# Mitsiu Alejandro Carreño Sarabia - E23S-18014

* Obtener los datos de Breast cancer 
* Entrenar un MLP para clasificar el objetivo
* Realizar una búsqueda de mallado para la mejor arquitectura
* Agrega a la búsqueda cualquier hiper-parámetro que quieras optimizar
* Generar un cuaderno con sus procedimientos documentados y tus conclusiones (importante)


https://keras.io/api/keras_tuner/tuners/grid/

## Carga de datos

In [1]:
import numpy as np
import pandas as pd
from sklearn.datasets import load_breast_cancer

data = load_breast_cancer()
X = pd.DataFrame(data.data, columns=data.feature_names)
y = pd.DataFrame(data.target, columns=["Type"])

In [2]:
print(data.feature_names)
print(y.shape)
print(X.shape)
print(data.DESCR)

['mean radius' 'mean texture' 'mean perimeter' 'mean area'
 'mean smoothness' 'mean compactness' 'mean concavity'
 'mean concave points' 'mean symmetry' 'mean fractal dimension'
 'radius error' 'texture error' 'perimeter error' 'area error'
 'smoothness error' 'compactness error' 'concavity error'
 'concave points error' 'symmetry error' 'fractal dimension error'
 'worst radius' 'worst texture' 'worst perimeter' 'worst area'
 'worst smoothness' 'worst compactness' 'worst concavity'
 'worst concave points' 'worst symmetry' 'worst fractal dimension']
(569, 1)
(569, 30)
.. _breast_cancer_dataset:

Breast cancer wisconsin (diagnostic) dataset
--------------------------------------------

**Data Set Characteristics:**

    :Number of Instances: 569

    :Number of Attributes: 30 numeric, predictive attributes and the class

    :Attribute Information:
        - radius (mean of distances from center to points on the perimeter)
        - texture (standard deviation of gray-scale values)
     

In [3]:
X.tail()

,mean radius,mean texture,mean perimeter,mean area,mean smoothness,mean compactness,mean concavity,mean concave points,mean symmetry,mean fractal dimension,...,worst radius,worst texture,worst perimeter,worst area,worst smoothness,worst compactness,worst concavity,worst concave points,worst symmetry,worst fractal dimension
564,21.56,22.39,142.00,1479.0,0.11100,0.11590,0.24390,0.13890,0.1726,0.05623,...,25.450,26.40,166.10,2027.0,0.14100,0.21130,0.4107,0.2216,0.2060,0.07115
565,20.13,28.25,131.20,1261.0,0.09780,0.10340,0.14400,0.09791,0.1752,0.05533,...,23.690,38.25,155.00,1731.0,0.11660,0.19220,0.3215,0.1628,0.2572,0.06637
566,16.60,28.08,108.30,858.1,0.08455,0.10230,0.09251,0.05302,0.1590,0.05648,...,18.980,34.12,126.70,1124.0,0.11390,0.30940,0.3403,0.1418,0.2218,0.07820
567,20.60,29.33,140.10,1265.0,0.11780,0.27700,0.35140,0.15200,0.2397,0.07016,...,25.740,39.42,184.60,1821.0,0.16500,0.86810,0.9387,0.2650,0.4087,0.12400
568,7.76,24.54,47.92,181.0,0.05263,0.04362,0.00000,0.00000,0.1587,0.05884,...,9.456,30.37,59.16,268.6,0.08996,0.06444,0.0000,0.0000,0.2871,0.07039


In [4]:
y.tail()

,Type
564,0
565,0
566,0
567,0
568,1


## Estandarización y creación de datasets

In [5]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

## Implementación de tuner

In [6]:
from tensorflow import keras
import keras_tuner

2023-10-11 23:38:47.315567: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-10-11 23:38:47.355824: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-10-11 23:38:47.356629: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-10-11 23:38:48.127516: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


Using TensorFlow backend


In [7]:
## Keras general model
'''
Generamos un modelo general de keras, que va a recibir valores dinamicos del tuner
'''
def define_model(units, activation, dropout, lr):
    model = keras.Sequential()
    model.add(
        keras.layers.Input(
            shape=(30,)
        )
    )
    model.add(
        keras.layers.Dense(
            units=units,
            activation=activation  # relu o softmax
        )
    )
    if dropout:
        model.add(
            keras.layers.Dropout(rate=0.2)
        )
    # Output layer
    model.add(
        keras.layers.Dense(1, activation="sigmoid")  
    )
    
    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=lr),
        loss="binary_crossentropy", 
        metrics=["accuracy"]
    )
    return model
    

## Tuner values
'''
Definir los hiperparametros 
* Neuronas en la primera capa oculta
* Función de activación en la primera capa oculta
* Evaluar si agregar capa de dropout o no
* Ajustar tasa de aprendizaje
'''
def build_model(hp):
    units = hp.Int("units", min_value=30/2, max_value=30*2, step=5)
    activation = hp.Choice("activation", ["relu", "softmax"])
    dropout = hp.Boolean("dropout")
    lr= hp.Float("lr", min_value=1e-4, max_value=1e-2, sampling="log")
    model = define_model(
        units=units, activation=activation, dropout=dropout, lr=lr
    )
    return model

In [8]:
tuner = keras_tuner.GridSearch(
    build_model,
    objective="val_accuracy",
    max_trials=10,
    executions_per_trial=3,
    directory="s09_tuner",
    project_name="proyecto_tuner"
)

In [9]:
tuner.search_space_summary()

Search space summary
Default search space size: 4
units (Int)
{'default': None, 'conditions': [], 'min_value': 15, 'max_value': 60, 'step': 5, 'sampling': 'linear'}
activation (Choice)
{'default': 'relu', 'conditions': [], 'values': ['relu', 'softmax'], 'ordered': False}
dropout (Boolean)
{'default': False, 'conditions': []}
lr (Float)
{'default': 0.0001, 'conditions': [], 'min_value': 0.0001, 'max_value': 0.01, 'step': None, 'sampling': 'log'}


In [10]:
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2)

## Búsqueda de mejores hiperparametros

In [11]:
tuner.search(X_train, y_train, validation_data=(X_val, y_val), epochs=50)

Trial 10 Complete [00h 00m 11s]
val_accuracy: 0.9560439586639404

Best val_accuracy So Far: 0.9743589758872986
Total elapsed time: 00h 02m 00s


In [12]:
mejor_modelo = tuner.get_best_models(num_models=1)[0]
mejor_hiperparametros = tuner.get_best_hyperparameters(num_trials=1)[0]
mejor_modelo.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 15)                465       
                                                                 
 dense_1 (Dense)             (None, 1)                 16        
                                                                 
Total params: 481 (1.88 KB)
Trainable params: 481 (1.88 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [13]:
mejor_hiperparametros.values

{'units': 15,
 'activation': 'relu',
 'dropout': False,
 'lr': 0.0012589254117941675}

## Entrenar red con mejores parametros e hiperparametros encontrados

In [14]:
mejor_modelo.fit(X_train, y_train, epochs=50, validation_data=(X_val, y_val))

Epoch 1/50
12/12 [==============================] - 1s 15ms/step - loss: 0.1187 - accuracy: 0.9698 - val_loss: 0.1177 - val_accuracy: 0.9890
Epoch 2/50
12/12 [==============================] - 0s 5ms/step - loss: 0.1110 - accuracy: 0.9698 - val_loss: 0.1123 - val_accuracy: 0.9890
Epoch 3/50
12/12 [==============================] - 0s 5ms/step - loss: 0.1046 - accuracy: 0.9725 - val_loss: 0.1085 - val_accuracy: 0.9780
Epoch 4/50
12/12 [==============================] - 0s 5ms/step - loss: 0.0987 - accuracy: 0.9780 - val_loss: 0.1047 - val_accuracy: 0.9780
Epoch 5/50
12/12 [==============================] - 0s 5ms/step - loss: 0.0934 - accuracy: 0.9780 - val_loss: 0.1015 - val_accuracy: 0.9780
Epoch 6/50
12/12 [==============================] - 0s 5ms/step - loss: 0.0891 - accuracy: 0.9808 - val_loss: 0.0985 - val_accuracy: 0.9670
Epoch 7/50
12/12 [==============================] - 0s 5ms/step - loss: 0.0848 - accuracy: 0.9835 - val_loss: 0.0959 - val_accuracy: 0.9670
Epoch 8/50
12/12 [=

## Evaluación con datos no vistos previamente

In [15]:
test_loss, test_accuracy = mejor_modelo.evaluate(X_test, y_test)
print(f"Perdida test: {test_loss:.4f}")
print(f"Precision test: {test_accuracy * 100 :.2f}%")

4/4 [==============================] - 0s 2ms/step - loss: 0.0862 - accuracy: 0.9825
Perdida test: 0.0862
Precision test: 98.25%


## Conclusiones
Creo que una herramienta como keras_tuner es una buena herramienta teniendo las nociones correctas de una red neuronal y los datos con los que se alimente, creo que tuner aplica una estrategía similar a una [busqueda por fuerza bruta](https://en.wikipedia.org/wiki/Brute-force_search) intentando muchas combinaciones e intentando encontrar la mejor dentro del espacio de búsqueda.
Definitivamente me ayudo a encontrar una tasa de aprendizaje que seguramente yo no hubiera encontrado con tanta precisión.       
Finalmente creo que es una buena herramienta a considerar para desarrollo de redes neuronales desde cero, y un buen punto de partida para comenzar a refinar la red.